In [7]:
df <- read.table("https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat")

In [8]:
colnames(df) <-  c('frequency', 		# 1. Частота, Гц.
                    'angle', 			# 2. Угол атаки, градусы.
                    'chord_length', 	# 3. Длина хорды профиля лопатки, м.
                    'fs_velocity', 		# 4. Скорость набегающего потока, м/с.
                    'ssd_thickness', 	# 5. Толщина вытеснения на выпуклой стороне (спинке) лопатки, м.
                    'sound_level') 		# 6. Уровень громкости, дБ.

In [9]:
head(df)

frequency,angle,chord_length,fs_velocity,ssd_thickness,sound_level
800,0,0.3048,71.3,0.00266337,126.201
1000,0,0.3048,71.3,0.00266337,125.201
1250,0,0.3048,71.3,0.00266337,125.951
1600,0,0.3048,71.3,0.00266337,127.591
2000,0,0.3048,71.3,0.00266337,127.461
2500,0,0.3048,71.3,0.00266337,125.571


In [10]:
fit <- lm(sound_level ~ frequency + angle + chord_length + fs_velocity + ssd_thickness, data=df)

In [11]:
summary(fit)


Call:
lm(formula = sound_level ~ frequency + angle + chord_length + 
    fs_velocity + ssd_thickness, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.480  -2.882  -0.209   3.152  16.064 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.328e+02  5.447e-01  243.87   <2e-16 ***
frequency     -1.282e-03  4.211e-05  -30.45   <2e-16 ***
angle         -4.219e-01  3.890e-02  -10.85   <2e-16 ***
chord_length  -3.569e+01  1.630e+00  -21.89   <2e-16 ***
fs_velocity    9.985e-02  8.132e-03   12.28   <2e-16 ***
ssd_thickness -1.473e+02  1.501e+01   -9.81   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 4.809 on 1497 degrees of freedom
Multiple R-squared:  0.5157,	Adjusted R-squared:  0.5141 
F-statistic: 318.8 on 5 and 1497 DF,  p-value: < 2.2e-16
